# Credmark Modeling Framework Example for Jupyter notebook
## Ledger Model

version: 2022.6.3

## Initialize

<div class="alert alert-block alert-info">
    <b>Note:</b> Change to a web3 provider you have in `params`/`chain_to_provider_url` below.
</div>

In [1]:
%reload_ext credmark.cmf.ipython

param = {'chain_id': 1,
 'block_number': None,
 'model_loader_path': ['../../credmark-models-py/models'],
 'chain_to_provider_url': {'1': 'http://192.168.68.122:10444'},
 'api_url': None,
 'use_local_models': None,
 'register_utility_global': True}

context, model_loader = %cmf param

## Example

### Contract - Uni V2

In [2]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
print(contract.functions.name().call(), contract.functions.symbol().call())

for addr in [contract.functions.token0().call(), contract.functions.token1().call()]:
    print(Token(address=addr).symbol)

Uniswap V2 UNI-V2
WETH
USDT


### Ledger Examples

- Get the max value group by `to`

In [3]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')

with contract.ledger.functions.Transfer as q:
    ret = (q.select(
        aggregates=[(q.VALUE.max_(), 'max_value')],
        group_by=[q.TO,q.TXN_BLOCK_NUMBER],
        order_by=q.field('max_value').dquote().desc(),
        where=q.TXN_BLOCK_NUMBER.gt(context.block_number - 2000000),
        limit=10
    ).to_dataframe())
display(ret)

,inp_to,txn_block_number,max_value
0,0x00b1a4E7F217380a7C9e6c12F327AC4a1D9B6A14,14138857,149783162423129570
1,0x00b1a4E7F217380a7C9e6c12F327AC4a1D9B6A14,14128554,21493808992440640
2,0x0aCd6bab83089f9ACa17E64bd13eA4502e5db533,14396337,7743549689439966
3,0x00b1a4E7F217380a7C9e6c12F327AC4a1D9B6A14,14097405,4736927601639810
4,0x00b1a4E7F217380a7C9e6c12F327AC4a1D9B6A14,14232262,2826632328075661
5,0x00b1a4E7F217380a7C9e6c12F327AC4a1D9B6A14,14434032,2762062321273503
6,0x00b1a4E7F217380a7C9e6c12F327AC4a1D9B6A14,14252590,975279293939838
7,0x87D4b3287C7644c02885e9301A56b56BC68ef3F1,13224497,836488387231461
8,0x17EC368352402EF554ea6D43AEF7d848C4D9884A,14186008,661189712558630
9,0xA0E7F672bc7BDB06EE542Fa2A8Dcf478F3c25F9C,14210339,548368413393905


- Get max amount0Out by `to`.

In [4]:
with contract.ledger.events.Swap as q:
    ret = (q.select(
        aggregates=[(q.AMOUNT0OUT.max_(), 'max_value')],
        group_by=[q.TO],
        order_by=q.field('max_value').dquote().desc(),
        where=q.EVT_BLOCK_NUMBER.gt(context.block_number - 2000000),
        limit=5)
    .to_dataframe())
display(ret)

,inp_to,max_value
0,0x56178a0d5F301bAf6CF3e1Cd53d9863437345Bf9,1911747785637249679360.0
1,0xBEEFBaBEeA323F07c59926295205d3b7a17E8638,1667946016386711814144.0
2,0x00000000003b3cc22aF3aE1EAc0440BcEe416B40,1509320889197135921152.0
3,0x000000000035B5e5ad9019092C665357240f594e,1132703857883090518016.0
4,0x000000000dFDe7deaF24138722987c9a6991e2D4,959324059003144600000


- Get max amount0Out and max amount1Out

In [5]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
with contract.ledger.events.Swap as q:
    ret = (q.select(
        aggregates=[(q.AMOUNT0OUT.max_(), 'max_amount0Out'), (q.AMOUNT1OUT.max_(), 'max_amount1Out')]
    ).to_dataframe())
display(ret)

,max_amount0Out,max_amount1Out
0,9.609250e+22,50000000000000


- Get the transaction hash containing the max amount0Out.

In [6]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
with contract.ledger.events.Swap as q:
    ret2 = (q.select(
        aggregates=[(q.AMOUNT0OUT.max_(), 'max_amount0Out')],
        group_by=[q.EVT_HASH],
        having=q.AMOUNT0OUT.max_().ge(ret.max_amount0Out[0]),
    ).to_dataframe())
display(ret2)

,evt_tx_hash,max_amount0Out
0,0x8bb8dc5c7c830bac85fa48acad2505e9300a91c3ff23...,9.609250e+22


- Get the transaction hash containing the max amount0Out or max amount1Out

For any column in `having` needs to be in `aggregates`.

In [7]:
contract = Contract(address='0x0d4a11d5EEaaC28EC3F61d100daF4d40471f1852')
with contract.ledger.events.Swap as q:
    ret3 = (q.select(
        aggregates=[(q.AMOUNT0OUT.max_(), 'max_amount0Out'),
                    (q.AMOUNT1OUT.max_(), 'max_amount1Out')],
        group_by=[q.EVT_HASH],
        having=q.AMOUNT0OUT.max_().ge(ret.max_amount0Out[0]).or_(
            q.AMOUNT1OUT.max_().ge(ret.max_amount1Out[0])),
    ).to_dataframe())

display(ret3)

,evt_tx_hash,max_amount0Out,max_amount1Out
0,0x0fc6d2ca064fc841bc9b1c1fad1fbb97bcea5c9a1b2b...,0.000000e+00,50000000000000
1,0x9ab36df290ab78f52f3e9be8c43a6b1d61ea80ec2f40...,0.000000e+00,50000000000000
2,0xc2cc773009ebaee06c1301a485794cbf0a56e657ac30...,0.000000e+00,50000000000000
3,0x89316cf5d24c82ca2dbbd48989d4c62e44bccaecdcf3...,0.000000e+00,50000000000000
4,0xdc4db0312149bc13607d07ec889ae96a8527866bbf40...,0.000000e+00,50000000000000
5,0x8bb8dc5c7c830bac85fa48acad2505e9300a91c3ff23...,9.609250e+22,0
6,0x789da3bb1e14a7ee5014e8150d7c28dc43bf40caa717...,0.000000e+00,50000000000000
7,0xe60b28f063036c9055f1ea92f09e3467d6e02cb90226...,0.000000e+00,50000000000000
8,0xa9e80d1214eb369ca443f4a025d7605914f758bc8103...,0.000000e+00,50000000000000
9,0x0f6dee0f75aa648c15f61bcc89ae86835a2c55fdd525...,0.000000e+00,50000000000000
